<a href="https://colab.research.google.com/github/muhafidz-ahmad/certificate-and-email-automation/blob/main/Automation_Certificate_Generator_and_Email_Sender.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Sertifikat Peserta Generator

## Import Data dan Inisialisasi Parameter

In [ ]:
import pandas as pd

nama_sheet = 'Final' #@param ['Peserta', '10 Besar', 'Final', 'Tim Soal', 'Uji Coba']
untuk = '10 Besar' #@param ['Juara 1', 'Juara 2', 'Juara 3', 'Juara 4', 'Juara 5', '10 Besar']
format_gambar = 'png' #@param ['png', 'jpg']
nama_font = 'CAVIARDREAMS_BOLDITALIC.TTF' #@param {type:'string'}
ukuran_font = 192 #@param {type:"integer"}

path = "/Sertifikat/"
data = pd.read_excel(path + 'Data.xlsx', sheet_name=nama_sheet)
data_nama = data['Nama']
data_email = data['Email']
juara = data['Juara']
data

## Login Email Pengirim

In [ ]:
from email.mime.text import MIMEText
from email.mime.application import MIMEApplication
from email.mime.multipart import MIMEMultipart
import os
import sys
from os.path import exists
import smtplib
from getpass import getpass

# Login
username = "email@domain.com"
pin = getpass("Masukan pin : ")
server = smtplib.SMTP('smtp.gmail.com:587')
server.starttls()
server.login(username, pin)

## Fungsi Pembuat dan Pengirim Sertifikat

In [ ]:
from PIL import Image,ImageDraw,ImageFont

font = ImageFont.truetype(path + nama_font, ukuran_font)

# Pembuatan Nama Pada Sertifikat 
save_folder = ''

def BuatSertif(nama):
  global save_folder
  if nama_sheet == 'Peserta':
    save_folder = path + 'Sertifikat Peserta/'
  elif nama_sheet == 'Tim Soal':
    save_folder = path + 'Sertifikat Tim Soal/'
  elif nama_sheet == 'Uji Coba':
    save_folder = path + 'Sertifikat Uji Coba/'
  elif nama_sheet == '10 Besar':
    save_folder = path + 'Sertifikat 10 Besar/'
  elif nama_sheet == 'Final' :
    save_folder = path + 'Sertifikat Final/'
  else:
    sys.exit('Tidak ada sheet', nama_sheet)

  sertif = untuk + '.{}'.format(format_gambar)

  save_folder = save_folder + 'Sertifikat {} {}.{}'.format(untuk, nama, format_gambar)

  img = Image.open(path + sertif)
  draw = ImageDraw.Draw(img)

  center_x, center_y = img.size
    
  w, h = draw.textsize(nama, font=font) #untuk mendapatkan ukuran pixel text

  # tulis nama di sertifikat
  draw.text(xy=((center_x - w + 454.385)/2, 769.067), text=nama, fill=(42,46,61), 
            font=font)
  
  img.save(save_folder)

  return save_folder
  
# Pengiriman Sertifikat Ke Email
def KirimSertif(nama, email, filename):
  sender = username

  #filename = save_folder

  msg = MIMEMultipart()
  msg['Subject'] = 'Sertifikat KOMET 2021'
  msg['From'] = username
  msg['Reply-to'] = username
  msg['To'] = email

  # That is what u see if dont have an email reader:
  msg.preamble = 'Multipart massage.\n'
      
  # Body
  body = """Terima kasih, Kak {} sudah ikut berpartisipasi dalam Kompetisi Matematika (KOMET) 2021 yang termasuk ke dalam rangkaian acara Mathematics Festival 2021
  
Tetap semangat, semoga sukses selalu.""".format(nama)

  part = MIMEText(body)
  msg.attach(part)

  # Attachment
  part = MIMEApplication(open(filename,"rb").read())
  part.add_header('Content-Disposition', 'attachment', filename = os.path.basename(filename))
  msg.attach(part)

  # Kirim email
  server.sendmail(msg['From'], msg['To'], msg.as_string())

  print("     Sertifikat di kirim ke", msg["To"])

## Jalankan Program

In [ ]:
# print("Sertifikat akan disimpan di", save_folder)
konfirmasi = input("Buat sertifikat? (y / n) ")
if konfirmasi == 'y':
  nomor = 1
  for nama, email, j in zip(data_nama, data_email, juara) :
    untuk = j
    file_sertif = BuatSertif(nama)
    print("{}. Sertifikat {}".format(nomor, untuk), nama, "berhasil dibuat")
    #KirimSertif(nama, email, file_sertif)
    nomor += 1
else:
  print("Sertifikat tidak jadi dibuat.")